In [1]:
!pip install prometheus-eval accelerate bitsandbytes --quiet

In [2]:
import matplotlib.pyplot as plt
import transformers
import pandas as pd
import numpy as np
import torch
import json
import sys
import os

# from dotenv import load_dotenv
from huggingface_hub import login

# load_dotenv()
# path = os.environ['DATA_PATH']
# os.environ["ACCELERATE_USE_TORCH_DEVICE"] = "true"
path = os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", "data", "ita")
hf_token_path = os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", "HF_TOKEN.txt")
with open(hf_token_path, "r") as f:
    hf_token = f.read().strip()

login(token=hf_token)
cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"

llama_out_fn = "Pizza_Language_&_Mandolino-hw2_ocr-llama.json"
minerva_out_fn = "Pizza_Language_&_Mandolino-hw2_ocr-minerva.json"


In [209]:
datasetRaw = {}

for i in [ocred_fn, cleaned_fn]:
    if i not in os.listdir(path):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(path, i)
    with open(file_path, 'r') as f:
        datasetRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [29]:
outRaw = {}

for i in [llama_out_fn, minerva_out_fn]:
    if i not in os.listdir(os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione")):
        print(f"ERROR 404 ! File {i} not Found...")

    file_path = os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", i)
    with open(file_path, 'r', encoding='utf-8') as f:
        outRaw[i.split('.')[0]] = json.load(f)
        f.close()

In [30]:
print(outRaw['Pizza_Language_&_Mandolino-hw2_ocr-llama']['1'])

I. Come andò che Maestro Ciliegia, Megnamc trovò un pezzo di legno che piangeva e rideva come un bambino. — C'era una volta.... — Un re! -diranno subito i miei piccoli lettori. — No, ragazzi, avete sbagliato. C'era una volta un pezzo di legno. Con era un legno di lusso, ma un semplice pezzo da catasta, di quelli che d'inverno si mettono nelle stufe e nei caminetti per accendere il fuoco e per riscaldare le stanze. Non so come andasse, ma il fatto gli è che un bel giorno' questo pezzo di legno capitò nella bottega di un vecchio falegname, il quale aveva nome mastr' Antonio, se non che tutti lo chiamavano maestro Ciliegia, per via della punta del sentì nna vocina sottile sottile. SUO naso, che era sempre lustra e paonazza, come una ciliegia matura. Appena maestro Ciliegia ebbe visto quel pezzo di legno, si rallegrò tutto; e dandosi una fregatina di mani per la contentezza, borbottò a mezza voce: — Questo legno è capitato a tempo; voglio servirmene per fare una gamba di tavolino. — Detto 

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

model_id = "Unbabel/M-Prometheus-7B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [246]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
    do_sample=True,
    temperature=0.1
)

Device set to use cuda:0


In [249]:
scores_dict = {
    'llama': {
                "paragraphs": [],
                "scores": []
            },
    'minerva': {
                "paragraphs": [],
                "scores": []
            }
}

In [250]:

for model_name in ['llama', 'minerva']:
    print(model_name)
    for paragraph in range(1, 8):
        prompt = f"""
            Valuta il testo ripulito da errori OCR generato dal modello rispetto alla corrispondente versione corretta, tenendo conto di correttezza, comprensibilità e somiglianza.
            Segui il seguente metro di valutazione:
            - 1: Pulizia completamente inaccettabile: Il testo non è stato ripulito e gli errori OCR persistono.
            - 2: Gravi errori semantici, omissioni o aggiunte sostanziali al testo originale.
            - 3: Pulizia parzialmente sbagliata, l'output ottenuto è superficiale. Contiene ancora errori, ma sono per lo più errori da un punto di vista semantico.
            - 4: Buona pulizia. L'outpt del testo è in gran parte corretta e fedele al testo, ma contiene ancora qualche errore minore da un punto di vista semantico. Il testo risulta fluente e comprensibile.
            - 5: Pulizia perfetta. Il testo è stato ripulito correttamente da errori OCR.

            Versione corretta: {datasetRaw['cleaned'][str(paragraph)]}
            Versione {model_name}: {outRaw[f'Pizza_Language_&_Mandolino-hw2_ocr-{model_name}'][str(paragraph)]}

            Rispondi solamente con il voto, seguendo il seguente formato per la risposta. 
            Valutazione: 
        """
        
        result = pipe(prompt)[0]["generated_text"]
        result = result.split("Valutazione: ")[-1].strip().split("\n")[0]
        scores_dict[model_name]['paragraphs'].append(str(paragraph))
        scores_dict[model_name]['scores'].append(result)
        print(f"{paragraph}: {result}")

llama
1: 2
2: 2
3: 1
4: 2
5: Voto: 3
6: 2
7: 3
minerva
1: 2
2: 2
3: 2
4: 2
5: 2
6: Voto: 2
7: 2


In [262]:
print(scores_dict)

{'llama': {'paragraphs': ['1', '2', '3', '4', '5', '6', '7'], 'scores': ['2', '2', '1', '2', '3', '2', '3']}, 'minerva': {'paragraphs': ['1', '2', '3', '4', '5', '6', '7'], 'scores': ['2', '2', '2', '2', '2', '2', '2']}}


In [ ]:
formatted_data = {
    model: [
        f"paragraph {p} : score {s}"
        for p, s in zip(content['paragraphs'], content['scores'])
    ]
    for model, content in scores_dict.items()
}

fname = "Pizza_Language_&_Mandolino-hw2_ocr-prometheus.json"
with open(os.path.join(os.getcwd(), "Desktop", "AI_R", "Formazione", fname), 'w', encoding='utf-8') as f:
    json.dump(formatted_data, f, indent=4)